In [43]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import synapse.ml.core
from synapse.ml.services import AnalyzeText
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 448e8de6-88bc-4b25-b437-175f14105a1a, 45, Finished, Available, Finished)

In [44]:
df =spark.read.format("delta").load("Tables/dbo/tbl_latest_news")

StatementMeta(, 448e8de6-88bc-4b25-b437-175f14105a1a, 46, Finished, Available, Finished)

In [45]:
model = (AnalyzeText()
.setTextCol("description")
.setKind("SentimentAnalysis")
.setOutputCol("response")
.setErrorCol("error"))

StatementMeta(, 448e8de6-88bc-4b25-b437-175f14105a1a, 47, Finished, Available, Finished)

In [46]:
result = model.transform(df)

StatementMeta(, 448e8de6-88bc-4b25-b437-175f14105a1a, 48, Finished, Available, Finished)

In [47]:
display(result)

StatementMeta(, 448e8de6-88bc-4b25-b437-175f14105a1a, 49, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 32cb2e39-5b32-4b8e-b90f-8a2b2e9ad075)

In [48]:


sentiment_df= result.withColumn("sentiment",result["response"].documents.sentiment)
display(sentiment_df)

StatementMeta(, 448e8de6-88bc-4b25-b437-175f14105a1a, 50, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d3eb3d1c-1577-4823-8468-33f4ea9145fb)

In [49]:
sentiment_df = sentiment_df.drop("error","response")
display(sentiment_df)

StatementMeta(, 448e8de6-88bc-4b25-b437-175f14105a1a, 51, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, eaf92d89-1a0e-4915-afca-b1a44931e3ed)

In [50]:
sentiment_df = sentiment_df.withColumn("datepublished",to_date(col("datepublished"),"dd-MMM-yyyy"))
display(sentiment_df)

StatementMeta(, 448e8de6-88bc-4b25-b437-175f14105a1a, 52, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1b4ae97d-8e77-4aff-94ac-fa5d14a39763)

In [51]:
from pyspark.sql.utils import AnalysisException
table_name = "Bing_Lakehouse.dbo.tbl_sentiment_Analysis"
try:
    sentiment_df.write.format("delta").saveAsTable(table_name)
except AnalysisException:
    print("table already exists")
    sentiment_df.createOrReplaceTempView("view_df_sentiment")
    spark.sql(f"""
    MERGE INTO {table_name} target_table 
    USING view_df_sentiment source_view
    on target_table.url = source_view.url
    when matched and 
    source_view.title <> target_table.title or 
    source_view.description <> target_table.description or 
    source_view.category <> target_table.category or 
    source_view.image <> target_table.image or 
    source_view.provider<>target_table.provider or 
    source_view.datepublished <> target_table.datepublished 
    then update set *  
    when not matched then insert *
    """)



StatementMeta(, 448e8de6-88bc-4b25-b437-175f14105a1a, 53, Finished, Available, Finished)

table already exists


In [52]:
%%sql
SELECT * from tbl_sentiment_analysis

StatementMeta(, 448e8de6-88bc-4b25-b437-175f14105a1a, 54, Finished, Available, Finished)

<Spark SQL result set with 37 rows and 8 fields>